In [10]:
import numpy as np
from scipy import linalg as lg

# ALS

In [11]:
def ALS(tensor, dims, rank=None, tol=1e-14, max_iter=100):
    N = len(dims)

    if rank is None or rank > min(dims):
        rank = min(dims)

    factors = [np.random.rand(dims[n], rank) for n in range(N)]

    eps_cur = tol + 1.0
    it = 0

    while it < max_iter and eps_cur > tol:
        eps_cur = 0.0

        for mode in range(N):
            Q = np.ones((1, rank))
            for m in range(N):
                if m == mode:
                    continue
                Q = lg.khatri_rao(Q, factors[m])

            axes = [i for i in range(N) if i != mode] + [mode]
            Tn = np.transpose(tensor, axes).reshape(-1, dims[mode])

            A_old = factors[mode]

            X = np.linalg.lstsq(Q, Tn, rcond=None)[0].T
            factors[mode] = X

            diff = np.linalg.norm(A_old - X)
            normA = np.linalg.norm(A_old)
            eps_cur = max(eps_cur, diff / (normA if normA else 1.0))

        it += 1

    return factors, it


# Testing

In [12]:

sizes = np.array((10, 10, 20))
T = np.zeros(sizes)
for i in range(sizes[0]):
    for j in range(sizes[1]):
        for k in range(sizes[2]):
            T[i, j, k] = i + j + k


rank = 10
factors, it = ALS(T, sizes, rank=rank, tol=1e-14, max_iter=100)


len_dim = len(sizes)
sweep_l = np.ones((1, rank))
for mode in range(len_dim):
    sweep_l = lg.khatri_rao(sweep_l, factors[mode])


recon_vec = np.sum(sweep_l, axis=1) 
orig_vec  = T.reshape(-1)

rel_err = np.linalg.norm(recon_vec - orig_vec) / np.linalg.norm(orig_vec)
print(f"ошибка: {rel_err:.3e}")

ошибка: 4.226e-16
